In [ ]:
# Import useful libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.cluster import KMeans

In [ ]:
# To avoid warnings in output cell
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# K-Means Clustering for CLuster Assignment Matrix
def KMeans_CAM(H, k):
    # Get the shape of the image
    rows, cols = H.shape

    # Apply KMeans Clustering
    kmeans = KMeans(n_clusters=k, random_state=0).fit(H)
    labels = kmeans.labels_

    # Convert to 8 bit values
    labels = np.uint8(labels)

    return labels

In [ ]:
# Normalize Matrix
def Normalize_Image(img):
    # Get the shape of the image
    rows, cols, ch = img.shape

    norm_img = np.zeros((rows, cols, ch), dtype=np.float32)

    for i in range(rows):
        for j in range(cols):
            for k in range(ch):
                norm_img[i, j, k] = img[i, j, k] / 255
    return norm_img

In [ ]:
# Generate Affinity Matrix
def affinity_matrix(sigma, lam, img):
    # Get the shape of the image
    rows, cols, ch = img.shape

    # Initialize the affinity matrix
    W = np.zeros((rows * cols, rows * cols), dtype=np.float32)

    # Compute the affinity matrix
    for i in range(rows):
        for j in range(cols):
            for k in range(rows):
                for l in range(cols):
                    # Compute the distance
                    dist = (img[i, j, 0] - img[k, l, 0]) ** 2  
                    dist += (img[i, j, 1] - img[k, l, 1]) ** 2
                    dist += (img[i, j, 2] - img[k, l, 2]) ** 2
                    x, y = i / 64, j / 64
                    x_, y_ = k / 64, l / 64
                    dist += (lam * ((x - x_) ** 2 + (y - y_) ** 2))

                    # Compute the affinity
                    W[i * rows + j, k * cols + l] = np.exp(-dist / (2 * (sigma ** 2)))
    return W

In [ ]:
# Generate Laplacian Matrix from Graph Adjascency Matrix
def Laplacian_Matrix(W):
    # Compute the degree matrix
    D = np.diag(np.sum(W, axis=1, dtype=np.float32))

    # Compute the Laplacian matrix
    L = D - W
    
    return L

In [ ]:
image_path = []

for path in image_path:
    # Read the image
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgRGB = img

    # Convert image to size of 64x64
    img = cv2.resize(img, (64, 64))
    imgRGB = cv2.resize(imgRGB, (64, 64))

    # Normalize the image
    normalize_img = Normalize_Image(img)

    # Get Affinity Matrix
    W = affinity_matrix(0.065, 0.04, normalize_img)
    
    # Get Laplacian Matrix
    L = Laplacian_Matrix(W)

    # Get the eigenvalues of the Laplacian matrix
    eigenvalues, eigenvectors = np.linalg.eig(L)

    # Sort the eigenvalues & Corresponding eigenvectors
    idx = eigenvalues.argsort()
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # Get the Cluster Assignment Matrix H
    H3 = eigenvectors[:, 0:3]
    H6 = eigenvectors[:, 0:6]

    # Cluster the Row of H using KMeans
    KMean_H3 = KMeans_CAM(H3, 3)
    KMean_H6 = KMeans_CAM(H6, 6)
    
    # Reshape the Cluster Assignment Matrix
    KMean_H3 = KMean_H3.reshape(64, 64)
    KMean_H6 = KMean_H6.reshape(64, 64)

    # Display 3 Image in a row
    # 1 : Original Image
    # 2 : Spectral Clustering K = 3
    # 3 : Spectral Clustering K = 6
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(imgRGB)
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(KMean_H3)
    plt.title('Spectral Clustering K = 3')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(KMean_H6)
    plt.title('Spectral Clustering K = 6')
    plt.axis('off')

    plt.show()
